# Summary by city of proximity analysis data (res9)

This notebook sumarizes in a csv pobtot and average max_times to each eje and amenity by city.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download available cities

In [2]:
mun_schema = 'metropolis'
mun_table = 'metro_gdf'

query = f"SELECT * FROM {mun_schema}.{mun_table}"
mun_gdf = aup.gdf_from_query(query, geometry_col='geometry')

city_list = mun_gdf.city.unique()

# Show
city_list

array(['Aguascalientes', 'Ensenada', 'Mexicali', 'Tijuana', 'La Paz',
       'Campeche', 'Laguna', 'Monclova', 'Piedras Negras', 'Saltillo',
       'Colima', 'Tecoman', 'Tapachula', 'Tuxtla', 'Chihuahua',
       'Delicias', 'Juarez', 'Parral', 'ZMVM', 'Durango', 'Celaya',
       'Guanajuato', 'Leon', 'Moroleon', 'San Francisco', 'Acapulco',
       'Chilpancingo', 'Pachuca', 'Tula', 'Tulancingo', 'Guadalajara',
       'Ocotlan', 'Vallarta', 'Tianguistenco', 'Toluca', 'Piedad',
       'Morelia', 'Zamora', 'Cuautla', 'Cuernavaca', 'Tepic', 'Monterrey',
       'Oaxaca', 'Tehuantepec', 'Puebla', 'Tehuacan', 'Teziutlan',
       'Queretaro', 'Cancun', 'Chetumal', 'Rio Verde', 'SLP', 'Culiacan',
       'Mazatlan', 'Guaymas', 'Hermosillo', 'Nogales', 'Villahermosa',
       'Victoria', 'Matamoros', 'Nuevo Laredo', 'Reynosa', 'Tampico',
       'Tlaxcala', 'Acayucan', 'Coatzacoalcos', 'Cordoba', 'Minatitlan',
       'Orizaba', 'Poza Rica', 'Veracruz', 'Xalapa', 'Merida',
       'Zacatecas'], dtype

## Create summary data

In [3]:
summary = pd.DataFrame()
i = 0

for city in city_list:
    
    print(f"Starting processing {city}, {i}/74")
    
    # Download proximity analysis data
    prox_schema = 'prox_analysis'
    prox_table = 'proximityanalysis_hexres9'
    query = f"SELECT * FROM {prox_schema}.{prox_table} WHERE \"city\" LIKE \'{city}\'"
    prox_gdf = aup.gdf_from_query(query, geometry_col='geometry')
    
    print(f"Downloaded proximity data for {city}")
    
    # Add city data
    summary.loc[i,'city'] = city
    
    # Add pobtot data
    summary.loc[i,'pobtot'] = prox_gdf['pobtot'].sum()
    
    # Avg data
    avg_list = ['max_escuelas', 'max_preescolar', 'max_primaria',
       'max_secundaria', 'max_servicios comunitarios', 'max_salud',
       'max_guarderías', 'max_asistencia social', 'max_comercio',
       'max_alimentos', 'max_personal', 'max_farmacias', 'max_hogar',
       'max_complementarios', 'max_entretenimiento', 'max_social',
       'max_actividad física', 'max_cultural','max_time']
    
    for col_name in avg_list:
        new_col_name = col_name.replace('max','avg')
        summary.loc[i,new_col_name] = prox_gdf[col_name].mean()
        
    i = i+1

Starting processing Aguascalientes, 0/74
Downloaded proximity data for Aguascalientes
Starting processing Ensenada, 1/74
Downloaded proximity data for Ensenada
Starting processing Mexicali, 2/74
Downloaded proximity data for Mexicali
Starting processing Tijuana, 3/74
Downloaded proximity data for Tijuana
Starting processing La Paz, 4/74
Downloaded proximity data for La Paz
Starting processing Campeche, 5/74
Downloaded proximity data for Campeche
Starting processing Laguna, 6/74
Downloaded proximity data for Laguna
Starting processing Monclova, 7/74
Downloaded proximity data for Monclova
Starting processing Piedras Negras, 8/74
Downloaded proximity data for Piedras Negras
Starting processing Saltillo, 9/74
Downloaded proximity data for Saltillo
Starting processing Colima, 10/74
Downloaded proximity data for Colima
Starting processing Tecoman, 11/74
Downloaded proximity data for Tecoman
Starting processing Tapachula, 12/74
Downloaded proximity data for Tapachula
Starting processing Tuxtl

## Show and save data

In [4]:
summary

,city,pobtot,avg_escuelas,avg_preescolar,avg_primaria,avg_secundaria,avg_servicios comunitarios,avg_salud,avg_guarderías,avg_asistencia social,...,avg_alimentos,avg_personal,avg_farmacias,avg_hogar,avg_complementarios,avg_entretenimiento,avg_social,avg_actividad física,avg_cultural,avg_time
0,Aguascalientes,1043173.0,26.590724,14.895257,15.677820,25.736201,40.757225,24.715144,19.074217,36.732291,...,8.112701,9.800499,16.177374,11.348914,10.184043,73.961675,28.415662,15.387824,73.639835,76.045215
1,Ensenada,396423.0,78.575715,14.920082,16.164558,77.595743,126.546214,19.804567,103.169164,124.008065,...,6.164440,12.154347,19.631260,16.598381,14.482764,226.768748,184.344469,22.147341,141.808426,228.137558
2,Mexicali,956287.0,31.196732,15.920821,16.101183,30.300229,100.998032,21.813474,43.333274,89.527121,...,5.593737,11.258720,18.838163,17.192771,16.857461,223.377365,58.261512,11.759853,220.321836,249.478649
3,Tijuana,2092757.0,39.333303,18.864521,19.192067,38.195434,56.568495,23.955085,38.170819,52.278324,...,8.403010,13.089477,29.618840,17.658549,16.882421,76.458991,51.037167,24.583852,74.225494,80.591474
4,La Paz,268798.0,30.597145,18.139794,19.023593,29.004315,186.836014,24.586064,177.684650,32.960353,...,9.879755,14.167475,27.914204,21.118497,16.456635,77.717011,48.585025,27.005334,72.907409,221.015240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Poza Rica,340284.0,19.404104,11.360580,12.159777,18.208899,69.840811,18.874601,65.052708,31.489021,...,5.164622,10.332281,17.161787,15.768408,12.195472,102.227978,20.917758,12.035050,101.607051,102.831396
70,Veracruz,825694.0,18.970534,12.071938,13.058052,17.498469,47.885814,17.960324,29.284870,42.194097,...,5.447609,10.061792,12.656844,10.423627,9.608979,93.552875,20.924554,13.233747,92.587275,97.565532
71,Xalapa,678137.0,19.867266,12.205467,12.875404,18.983215,48.708667,23.524162,28.579543,34.908924,...,5.217003,8.058532,15.608250,11.255911,9.543916,80.508890,27.073589,14.477565,78.256021,83.655979
72,Merida,1235042.0,33.486102,14.007767,14.270565,32.042883,44.671542,21.465857,27.846467,35.314818,...,5.611911,8.543722,17.424738,11.977815,9.416006,78.679152,26.475065,9.589384,78.011063,86.217528


In [5]:
summary.to_csv("../../data/external/temporal_fromjupyter/proxdata_avgsummary.csv")